In [42]:
#HERE YOU FIND ALL THE LIBRARYS FOR THIS PROJECT
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler

#this librarys are for the gpu usage
import tensorflow as tf
from keras.utils.training_utils import multi_gpu_model
from tensorflow.python.client import device_lib

In [43]:
"""
Setup multi GPU usage

Example usage:
model = Sequential()
...
multi_model = multi_gpu_model(model, gpus=num_gpu)
multi_model.fit()

About memory usage:
https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
"""

# IMPORTANT: Tells tf to not occupy a specific amount of memory
from keras.backend.tensorflow_backend import set_session  
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU  
sess = tf.Session(config=config)  
set_session(sess)  # set this TensorFlow session as the default session for Keras.


# getting the number of GPUs 
def get_available_gpus():
   local_device_protos = device_lib.list_local_devices()
   return [x.name for x in local_device_protos if x.device_type    == 'GPU']
num_gpu = len(get_available_gpus())
print('Amount of GPUs available: %s' % num_gpu)

Amount of GPUs available: 4


In [44]:
#SELECT THE DATA TO USE
df = pd.read_csv('//datc//opschaler//combined_gas_smart_weather_dfs//processed//all_dwellings_combined_hour.csv', delimiter='\t', parse_dates=['datetime'])
df = df.set_index(['datetime'])
df = df.dropna()

df['gasPower'] = df['gasPower']*10
df['T-1'] = df['T'].shift(1)

In [45]:
#RESAMPLE TO ONE DAY DATA
df_day = df.resample('1D').mean()
df_day = df_day.dropna()

In [46]:
df_day['hour'] = df_day.index.hour #create column containing the hour
df_day['dayofweek'] = df_day.index.dayofweek #Create column containing the day of the week

columns_to_cat = ['hour', 'dayofweek']
df_day[columns_to_cat] = df_day[columns_to_cat].astype('category') # change datetypes to category

df_day = pd.get_dummies(df_day, columns=columns_to_cat) # One hot encoding the categories

In [47]:
df_day.head()

eMeter  eMeterReturn    eMeterLow  eMeterLowReturn  \
datetime                                                              
2017-02-22  1556.361003           0.0  1803.676799              0.0   
2017-02-23  1558.435883           0.0  1805.226395              0.0   
2017-02-24  1561.837229           0.0  1806.231979              0.0   
2017-02-25  1564.963013           0.0  1808.506632              0.0   
2017-02-26  1564.963013           0.0  1817.410950              0.0   

                ePower  ePowerReturn     gasMeter          DD          DR  \
datetime                                                                    
2017-02-22  185.730438           0.0  1678.554576  243.831944  444.333333   
2017-02-23  173.374907           0.0  1685.894847  239.148194  265.916667   
2017-02-24  246.992340           0.0  1694.890589  278.093543   45.766667   
2017-02-25  150.144606           0.0  1698.313894  202.265418   14.750000   
2017-02-26  679.171475           0.0  1706.148254  208.350207   82.633334   

                   FF     ...       gasPower        T-1  hour_0  dayofweek_0  \
datetime                  ...                                                  
2017-02-22   8.236250     ...       2.562459  10.462500       1            0   
2017-02-23  11.608438     ...       4.857127   8.737917       1            0   
2017-02-24   5.032826     ...       2.768300   4.901806       1            0   
2017-02-25   6.502271     ...       0.171661   5.321389       1            0   
2017-02-26   7.540049     ...       5.030009   8.603333       1            0   

            dayofweek_1  dayofweek_2  dayofweek_3  dayofweek_4  dayofweek_5  \
datetime                                                                      
2017-02-22            0            1            0            0            0   
2017-02-23            0            0            1            0            0   
2017-02-24            0            0            0            1            0   
2017-02-25            0            0            0            0            1   
2017-02-26            0            0            0            0            0   

            dayofweek_6  
datetime                 
2017-02-22            0  
2017-02-23            0  
2017-02-24            0  
2017-02-25            0  
2017-02-26            1  

[5 rows x 32 columns]